In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

d:\SWE\rate-my-professor\rag\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name = "rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [2]:
import json
data = json.load(open("reviews.json"))


In [3]:
processed_data = []
client = OpenAI()

for review in data["reviews"]:
    response = client.embeddings.create(
        input = review['review'],
        model = "text-embedding-3-small",       
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id":review['professor'],
        "metadata":{
            "review":review['review'],
            "subject":review['subject'],
            "stars":review['stars'],
        },
    })

In [9]:
processed_data[0]

{'values': [-0.02576311,
  -0.0028632958,
  0.02134695,
  0.026614891,
  0.005906776,
  0.00470773,
  0.007633271,
  0.043820873,
  0.0018083974,
  0.009055091,
  0.019079901,
  0.0056086527,
  0.0069321897,
  0.009127164,
  -0.018123286,
  0.05134276,
  -0.009428564,
  -0.0093106255,
  0.020390335,
  0.061695177,
  0.04877431,
  0.009854455,
  0.012055982,
  -0.038317055,
  -0.0390509,
  -0.025802422,
  0.022172524,
  0.00095825404,
  0.030061329,
  -0.021713872,
  0.056872785,
  0.0025864667,
  -0.007698793,
  -0.006024715,
  -0.0581308,
  0.03708525,
  0.025396189,
  0.027047334,
  0.020717943,
  -0.015961071,
  -0.0043965024,
  -0.0031646953,
  -0.0048518777,
  0.004835497,
  0.020573795,
  -0.0033006526,
  0.0061655864,
  -0.0056381375,
  0.05330841,
  0.030166164,
  -0.030480668,
  -0.0012203406,
  0.011997012,
  -0.026863873,
  -0.03315395,
  -0.018673668,
  -0.0060705803,
  0.018844023,
  0.035643775,
  -0.0065325075,
  0.061537925,
  0.0012031412,
  -0.011479392,
  -0.02345674

In [7]:
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespae = "ns1"
)

{'upserted_count': 20}